In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Importing Libraries
!pip3 install graphviz
!pip3 install dask
!pip install "dask[complete]" 
!pip3 install toolz
!pip3 install cloudpickle
# https://www.youtube.com/watch?v=ieW3G7ZzRZ0
# https://github.com/dask/dask-tutorial
# please do go through this python notebook: https://github.com/dask/dask-tutorial/blob/master/07_dataframe.ipynb
import dask.dataframe as dd#similar to pandas

import pandas as pd#pandas to create small dataframes 

# if this doesnt work refere install_folium.JPG in drive
import folium #open street map

# unix time: https://www.unixtimestamp.com/
import datetime #Convert to unix time

import time #Convert to unix time

# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays

# matplotlib: used to plot graphs
import matplotlib
# matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  

# this lib is used while we calculate the stight line distance between two (lat,lon) pairs in miles
!pip install gpxpy
import gpxpy.geo #Get the haversine distance

from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os

import warnings
warnings.filterwarnings("ignore")

import datetime
import time
from IPython.display import display


     |████████████████████████████████| 125 kB 13.4 MB/s 
     |████████████████████████████████| 786 kB 51.1 MB/s 
     |████████████████████████████████| 779 kB 54.3 MB/s 
     |████████████████████████████████| 778 kB 65.2 MB/s 
     |████████████████████████████████| 776 kB 48.7 MB/s 
     |████████████████████████████████| 769 kB 54.3 MB/s 
     |████████████████████████████████| 766 kB 53.7 MB/s 
     |████████████████████████████████| 1.0 MB 48.4 MB/s 
     |████████████████████████████████| 722 kB 48.3 MB/s 
     |████████████████████████████████| 722 kB 49.8 MB/s 
     |████████████████████████████████| 715 kB 52.4 MB/s 
     |████████████████████████████████| 705 kB 33.4 MB/s 
     |████████████████████████████████| 699 kB 48.0 MB/s 
     |████████████████████████████████| 696 kB 43.8 MB/s 
     |████████████████████████████████| 684 kB 51.1 MB/s 
     |████████████████████████████████| 679 kB 43.2 MB/s 
     |████████████████████████████████| 675 kB 50.5 MB/s 
     |████████

In [ ]:
cleaned_data = pd.read_csv(r'/content/gdrive/MyDrive/New_York_Data/clean/clean_yellow_tripdata_2016-01_first_week.csv')#, assume_missing=True)#.iloc[:, 1:4]

In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Function Wrappers for Data Preprocessing / Clustering / Visualization

(See the main notebook "Taxi_Prediction.ipynb" for comments on this part)

In [ ]:
def unix2datetime(unx):
  return datetime.datetime.utcfromtimestamp(int(unx))

def datetime2unix(dt):
  return time.mktime(dt.timetuple())

def make_clusters(df, n_clusters):
  coords = df[['pickup_latitude', 'pickup_longitude']].values
  kmns = MiniBatchKMeans(n_clusters=n_clusters, batch_size=10000,random_state=0).fit(coords)
  df['pickup_cluster'] = kmns.predict(coords)
  return kmns


def make_bins(df, bin_mins):
  min_unix = df['pickup_times'].min()
  min_dt = unix2datetime(min_unix)
  year, month = min_dt.year, min_dt.month
  start_dt = datetime.date(year, month, 1)
  start_unix = datetime2unix(start_dt)

  period = bin_mins * 60
  bins = []
  for pu in df['pickup_times'].values:
    bins.append(int((pu - start_unix) // period))

  df['pickup_bins'] = bins

In [ ]:
def visualize_cluster_centers_map(centers):
  cluster_len = len(cluster_centers)
  map_osm = folium.Map(location=[40.734695, -73.990372], tiles='Stamen Toner')
  for i in range(cluster_len):
      folium.Marker(list((cluster_centers[i][0],cluster_centers[i][1])), popup=(str(cluster_centers[i][0])+str(cluster_centers[i][1]))).add_to(map_osm)
  display(map_osm)

def plot_clusters(df):
  city_long_border = (-74.03, -73.75)
  city_lat_border = (40.63, 40.85)
  fig, ax = plt.subplots(figsize=(15,15),ncols=1, nrows=1)
  ax.scatter(df.pickup_longitude.values[:100000], df.pickup_latitude.values[:100000], s=10, lw=0,
                c=df.pickup_cluster.values[:100000], cmap='tab20', alpha=0.2)
  ax.set_xlim(city_long_border)
  ax.set_ylim(city_lat_border)
  ax.set_xlabel('Longitude')
  ax.set_ylabel('Latitude')
  plt.show()

In [ ]:
def df2numpy(df, num_clusters):
  num_bins = df['pickup_bins'].max() + 1
  data = np.zeros((num_bins,num_clusters))
  for i in range(num_bins):
    cur_bin = df[df['pickup_bins'] == i]['pickup_cluster']
    for pu in cur_bin:
      data[i][pu] += 1
  return data

def save_data(data, year, month, num_clusters, bin_mins, save_dir="/content/gdrive/MyDrive/New_York_Data/Clustered/"):
  if month <= 10:
    month = "0" + str(month)
  name = "{}-{}_{}_{}.npy".format(year, month, num_clusters, bin_mins)
  save_path = os.path.join(save_dir, name)
  if os.path.exists(save_path):
    print("{} already exists, skipping...".format(save_path))
    return save_path
  with open(save_path, "wb") as f:
    np.save(f, data)
  return save_path

In [ ]:
def preprocess2npy(df, year, month, num_clusters, bin_mins, save_dir="/content/gdrive/MyDrive/New_York_Data/Clustered/"):
  print("Making clusters...")
  kmns = make_clusters(df, num_clusters)
  print("Making bins...")
  make_bins(df, bin_mins)
  data = df2numpy(df, num_clusters)
  print("Saving...")
  save_path = save_data(data, year, month, num_clusters, bin_mins, save_dir)
  print("Saved to", save_path)
  return save_path

In [ ]:
cleaned_data = pd.read_csv(r'/content/gdrive/MyDrive/New_York_Data/clean/clean_yellow_tripdata_2016-01_first_week.csv')

In [ ]:
preprocess2npy(cleaned_data, 2016, 1, 1000, 60)

Making clusters...
Making bins...
Saving...
Saved to /content/gdrive/MyDrive/New_York_Data/Clustered/2016-01_1000_60.npy


'/content/gdrive/MyDrive/New_York_Data/Clustered/2016-01_1000_60.npy'

In [ ]:
kmns = make_clusters(cleaned_data, 1000)

In [ ]:
with open("/content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-01_first_week.npy", "wb") as f:
  np.save(f, kmns.cluster_centers_)

##Install Torch-Geometric Dependencies

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-geometric
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.9.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 10.4 MB 13.6 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 3.7 MB 10.9 MB/s 
     |████████████████████████████████| 308 kB 16.3 MB/s 
     |████████████████████████████████| 407 kB 43.3 MB/s 
     |████████████████████████████████| 45 kB 4.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.1-py3-none-any.whl size=513822 sha256=a1de788a2ac54996fdf959924832a41f96b4da52c2fd215e9d376cab2e999e04
  Stored in directory: /root/.cache/pip/wheels/78/3d/42/20589db73c66b5109fb93a0c5743edfd6ab5ca820a52afacfc
Successfully built torch-geometric
Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 1.6 MB 3.2 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 870 kB 14.8 MB/s 


## Defining the G-CNN 

In [ ]:
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.nn import radius_graph
from torch_geometric.utils import add_self_loops
from sklearn.metrics.pairwise import rbf_kernel

class TaxiGraphDataset(InMemoryDataset):
  def __init__(self, device, transform=None, pre_transform=None):
    self.window_size = 5
    self.shuffle_dataset = False
    super(TaxiGraphDataset, self).__init__("/content/gdrive/MyDrive/New_York_Data/Clustered", transform, pre_transform)
    self.data, self.slices = torch.load(self.processed_paths[0])
    self.data.to(device)

  @property
  def raw_file_names(self):
    return ["/content/gdrive/MyDrive/New_York_Data/Clustered/2016-01_1000_60.npy"]

  @property
  def processed_file_names(self):
    return ['data.pt']

  def download(self):
    pass

  def process(self):
    with open("/content/gdrive/MyDrive/New_York_Data/Clustered/2016-01_1000_60.npy", 'rb') as f:
      data_list = []
      np_data = np.load(f)
      #print(shape(np_data))
      np_data = np.stack((np.roll(np_data, i, axis=0) for i in range(-1,self.window_size)), axis=2)
      #print(shape(np_data))
      indices = list(range(np_data.shape[0]))
      if self.shuffle_dataset:
        np.random.seed(self.random_seed)
        np.random.shuffle(indices)

    with open("/content/gdrive/MyDrive/New_York_Data/Clustered/cluster_centers_2016-01_first_week.npy", "rb") as f:
      cluster_centers = np.load(f, allow_pickle=True)

      clusterpos_x = cluster_centers[:,0] - np.mean(cluster_centers[:,0])
      clusterpos_y = cluster_centers[:,1] - np.mean(cluster_centers[:,1])

      clusterpos_x /= np.std(clusterpos_x)
      clusterpos_y /= np.std(clusterpos_y)
      clusterpos = np.stack((clusterpos_x, clusterpos_y), axis=1)

      position_embedding = np.repeat(rbf_kernel(clusterpos, gamma=1000)[np.newaxis,:,:], len(np_data), axis=0)

      #clusterpos_x = np.repeat(np.expand_dims((clusterpos_x), axis=(0,2)), 1009, axis=0)
      #clusterpos_y = np.repeat(np.expand_dims((clusterpos_y), axis=(0,2)), 1009, axis=0)
      #print(shape(clusterpos_x))
      data = np.concatenate((np_data, 50*position_embedding), axis=2)
      #print(shape(data))

    for i in indices:
      pos = torch.tensor(clusterpos)
      edges = radius_graph(x=pos, r=0.05)
      print(edges.shape)
      #edges, _ = add_self_loops(edges, num_nodes=pos.size(0))
      data_list.append(Data(edge_index=edges, x=torch.tensor(data[i], dtype=torch.float32)))
      
    if self.pre_filter is not None:
      data_list = [data for data in data_list if self.pre_filter(data)]

    if self.pre_transform is not None:
      data_list = [self.pre_transform(data) for data in data_list]

    data, slices = self.collate(data_list)
    torch.save((data, slices), self.processed_paths[0])   

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
def train_test_split(ds, train_split=0.8, batch_size=4):
  dataset_size = len(ds)
  indices = list(range(dataset_size))
  index_split = int(train_split * dataset_size)
  train_indices, test_indices = indices[:index_split], indices[index_split:]
  train_sampler = SubsetRandomSampler(train_indices)
  test_sampler = SubsetRandomSampler(test_indices)

  train_loader = DataLoader(ds, batch_size=batch_size,
                            sampler=train_sampler)
  test_loader = DataLoader(ds, batch_size=batch_size,
                            sampler=test_sampler)
  
  return (train_loader, test_loader)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

ds = TaxiGraphDataset(device)
train_loader, test_loader = train_test_split(ds, train_split=0.8)

cpu


Processing...
/usr/local/lib/python3.7/dist-packages/torch_geometric/data/dataset.py:88: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  self._process()


torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size([2, 1588])
torch.Size

Done!
/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1005, 1)
        #for p in self.conv1.parameters():
          #torch.nn.init.zeros_(p.data)
        #self.conv2 = GCNConv(32, 1)
        #self.conv3 = GCNConv(30, 1)
        #for p in self.conv2.parameters():
          #torch.nn.init.zeros_(p.data)

    def forward(self, data):
        x, edge_index = data.x[:,1:], data.edge_index
        #print(x)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.conv3(x, edge_index)
        #x = F.relu(x)
        return x[:, 0]

In [ ]:
model = GCN().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-3)

#model.train()
for epoch in range(200):
  for batch in train_loader:
    #print(batch.edge_index)
    #print("x")
    #print(batch.x[:,0])
    #print(batch.x)
    y = batch.x[:,0]
    #print("y")
    #print(y)
    pred = model(batch)
    #print("y: ", y)
    #print(y.dtype)
    #print("x: ", batch.x)
    #print("pred: ", pred)
    #print(torch.abs((pred - y) / (y+0.5)))
    #mape_loss = torch.mean(torch.abs((pred - y) / (y+1e-6) ) )  
    loss =  F.mse_loss(pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  losses = []
  for batch in test_loader:
    y = batch.x[:,0]
    pred = model(batch)
    #print(y)
    #print(pred)
    mape = err_metric(y.cpu().detach().numpy(), pred.cpu().detach().numpy())
    losses.append(mape.item())

    #print(losses)

  test_mape_loss =  np.array(losses).mean()
  print("Epoch: ", epoch, "MAPE Loss: ", test_mape_loss)
    #for p in model.parameters():
    #print(p.name, p.data)

Epoch:  0 MAPE Loss:  0.6514700591089748
Epoch:  1 MAPE Loss:  1.8281544705769934
Epoch:  2 MAPE Loss:  0.5143207929532819
Epoch:  3 MAPE Loss:  0.48627718526587915
Epoch:  4 MAPE Loss:  0.4995847202727713
Epoch:  5 MAPE Loss:  0.45312332418855916
Epoch:  6 MAPE Loss:  0.45777787873634473
Epoch:  7 MAPE Loss:  0.4653333363602048
Epoch:  8 MAPE Loss:  0.4638445983134301
Epoch:  9 MAPE Loss:  0.46138657639474306
Epoch:  10 MAPE Loss:  0.4563547676603614
Epoch:  11 MAPE Loss:  0.4552096655377596
Epoch:  12 MAPE Loss:  0.4584844044395677
Epoch:  13 MAPE Loss:  0.43576014243986555
Epoch:  14 MAPE Loss:  0.45110332034549405
Epoch:  15 MAPE Loss:  0.45157574497132047
Epoch:  16 MAPE Loss:  0.4507356727877252
Epoch:  17 MAPE Loss:  0.4468139488515799
Epoch:  18 MAPE Loss:  0.43013035572588504
Epoch:  19 MAPE Loss:  0.4430583351944366
Epoch:  20 MAPE Loss:  0.437923111748019
Epoch:  21 MAPE Loss:  0.44170510333854796
Epoch:  22 MAPE Loss:  0.4503388301040174
Epoch:  23 MAPE Loss:  0.47901627557

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import mean_absolute_error
def err_metric(y_true, y_pred):
  err = mean_absolute_error(y_true, y_pred) / (np.sum(y_true) / len(y_true))
  return err

In [ ]:
losses = []
for batch in test_loader:
  y = batch.x[:,0]
  pred = model(batch)
  #print(y)
  #print(pred)
  mape = err_metric(y.cpu().detach().numpy(), pred.cpu().detach().numpy())
  losses.append(mape.item())

#print(losses)

test_mse_loss =  np.array(losses).mean()
print(test_mse_loss)

0.41249619189620945
